# <b><span style='color:#0B2F9F'>Latar Belakang</span></b>

Di sebuah sudut kota Jakarta, telah didirikan sebuah restoran pizza bernama DQPizza. Selama satu tahun terakhir, telah dilakukan pencatatan atas setiap transaksi yang terjadi. Mulai dari pemesanan menu hingga pembayaran pelanggan. Banyak data telah dikumpulkan dan dicatat. Namun, belum dilakukan pemanfaatan optimal terhadap informasi yang tersedia. Pemilik restoran menginginkan agar data ini dapat dianalisis guna menemukan ruang untuk perbaikan dan melakukan efisiensi operasional sehingga diharapkan dapat meningkatkan penjualan pizza, menekan biaya operasional dan meningkatkan customer experience


# <b><span style='color:#0B2F9F'>Set up</span></b>

In [ ]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

In [ ]:
# Inisialisasi project_id dan dataset_id
project_id = 'dqlab-475713'
dataset_id = 'dq_pizza'

# Buat BigQuery client
client = bigquery.Client(project = project_id)
dataset_ref = bigquery.Dataset(f'{project_id}.{dataset_id}')
dataset = client.create_dataset(dataset_ref, exists_ok = True)

# <b><span style='color:#0B2F9F'>Data Digunakan</span></b>

In [ ]:
%%bigquery --project {project_id} --verbose

CREATE TABLE IF NOT EXISTS dq_pizza.tbl_all_transaction AS
WITH temp_order AS (
  SELECT
    order_id,
    customer_id,
    order_maker_id,
    PARSE_DATE('%m/%d/%Y', order_date) AS order_date,
    PARSE_TIME('%H:%M:%S', order_time) AS order_time,
    PARSE_TIME('%H:%M:%S', completion_time) AS completion_time,
    is_complain,
    complain_detail
  FROM `dqlab-9876543.dq_pizza.orders`
  WHERE order_id IS NOT NULL
), temp_order_detail AS (
  SELECT
    order_details_id,
    order_id,
    pizza_id,
    quantity
  FROM `dqlab-9876543.dq_pizza.order_details`
  WHERE order_details_id IS NOT NULL
), temp_pizza AS (
  SELECT DISTINCT
    pizza_id,
    pizza_type_id,
    size AS pizza_size,
    CAST(REPLACE(price, 'IDR', '') AS FLOAT64) AS pizza_price,
    CAST(REPLACE(production_cost, 'IDR', '') AS FLOAT64) AS pizza_production_cost
  FROM `dqlab-9876543.dq_pizza.pizzas`
), temp_pizza_type AS (
  SELECT DISTINCT
    pizza_type_id,
    UPPER(name) AS pizza_name,
    UPPER(category) AS pizza_category,
    ingredients AS pizza_ingredients
  FROM `dqlab-9876543.dq_pizza.pizza_types`
), temp_customer AS (
  SELECT DISTINCT
    customer_id,
    customer_name,
    gender AS customer_gender,
    DATE_DIFF(CURRENT_DATE(), CAST(birth_date AS DATE), YEAR) AS customer_age
  FROM `dqlab-9876543.dq_pizza.customers`
)
  SELECT
    o.order_id,
    od.order_details_id,
    od.pizza_id,
    o.customer_id,
    o.order_maker_id,
    o.order_date,
    c.customer_name,
    c.customer_gender,
    c.customer_age,
    pt.pizza_name,
    pt.pizza_category,
    p.pizza_size,
    p.pizza_price,
    p.pizza_production_cost,
    od.quantity,
    pt.pizza_ingredients,
    o.order_time,
    o.completion_time,
    o.is_complain,
    o.complain_detail,
    CURRENT_TIMESTAMP() AS created_date
  FROM temp_order AS o
  INNER JOIN temp_order_detail AS od ON o.order_id = od.order_id
  LEFT JOIN temp_pizza AS p ON od.pizza_id = p.pizza_id
  LEFT JOIN temp_pizza_type pt ON p.pizza_type_id = pt.pizza_type_id
  LEFT JOIN temp_customer c ON o.customer_id = c.customer_id

## **Area 1 - Performance Summary**

_Bagaimana tren jumlah transaksi dan total pendapatan penjualan pizza di DQPizza setiap bulan selama periode 2024? Adakah pola menarik yang kamu temukan?_



In [ ]:
%%bigquery summary_trx_per_month --project {project_id} --verbose

SELECT
  EXTRACT(MONTH FROM order_date) AS month,
  FORMAT_DATE('%B', order_date) AS month_name,
  COUNT(DISTINCT order_id) AS total_trx,
  ROUND(SUM(pizza_price * quantity), 2) AS total_revenue
FROM dq_pizza.tbl_all_transaction
WHERE EXTRACT(YEAR FROM order_date) = 2024
GROUP BY month, month_name
ORDER BY month

In [ ]:
# Tampilkan hasilnya
display(summary_trx_per_month)

In [ ]:
#@title ***DQPizza Performance Summary in 2024*** {display-mode: 'form'}

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows = 2,
    cols = 1,
    shared_xaxes = True,
    row_heights = [0.50, 0.75],
    vertical_spacing = 0.01
)

# Line chart pada subplot 1
fig.add_trace(
    px.line(
        summary_trx_per_month,
        x = 'month_name',
        y = 'total_trx',
        markers = True,
        color_discrete_sequence=['#03c03c'],
        labels = dict(
            x = 'month_name',
            y = 'total_trx'
        ),
        line_shape='linear'
    ).data[0],
    row = 1,
    col = 1
)

for i in [0, 11]:
    last_month_trx = summary_trx_per_month.loc[i, 'month_name']
    last_value_trx = summary_trx_per_month.loc[i, 'total_trx']
    if(i == 11):
        add_y = -100
    else:
        add_y = 100
    fig.add_annotation(
        x = last_month_trx,
        y = last_value_trx + add_y,
        text = f'<b>{last_month_trx}</b><br>Total Transaction : {last_value_trx}',
        showarrow = False,
        font = dict(
            color='#03c03c',
            family = "sans serif",
            size = 15,
        )
    )

fig.update_layout(
    yaxis = dict(
        showline = False,
        showgrid = False,
        showticklabels = False,
    )
)

# Add annotation for the maximum value with distance
max_month_trx = summary_trx_per_month.loc[summary_trx_per_month['total_trx'].idxmax(), 'month_name']
max_value_trx = summary_trx_per_month['total_trx'].max()

fig.add_annotation(
    x = max_month_trx,
    y = max_value_trx + 100,
    text = f'<b>{max_month_trx}</b><br>Best Transaction : {max_value_trx}',
    showarrow = False,
    font = dict(
        color='#02862a',
        family = "sans serif",
        size = 15,
    )
)

# Add line chart to the second row
fig.add_trace(
    px.bar(
        summary_trx_per_month,
        x = 'month_name',
        y = 'total_revenue',
        color_discrete_sequence=['#03c03c'],
        labels = dict(
            x = 'month_name',
            y = 'total_revenue'
          ),
          text = ['IDR ' + str(round(value / 1000000, 1)) + 'jt' for value in summary_trx_per_month['total_revenue']]
        ).data[0],
        row = 2,
        col = 1
    )

    # Update layout
fig.update_layout(
    height = 600,
    width = 1200,
    bargap = 0.05,
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    title = {
        'text': '<b>DQPizza Performance Summary in 2024</b>',
        'font': {'color': '#02862a', 'size': 20},
        'x' : 0.085
    }
)

"""
    fig.update_traces(
        marker = dict(
            color= ['#02862a' if val == 1 else '#03c03c' for val in summary_trx_per_month['rank_revenue']]
        )
    )
"""

fig.update_xaxes(showgrid=False)
fig.update_yaxes(
    showgrid  = False,
    visible = False,
    tickfont_color = "white"
)

fig.show()

**Kesimpulan :**     

Performa penjualan DQPizza sepanjang 2024 menunjukkan stabilitas dengan basis pelanggan yang loyal. Puncak penjualan pada bulan Juli membuka peluang kampanye besar di pertengahan tahun, sementara performa terendah di Desember menegaskan perlunya strategi promosi yang lebih agresif pada periode akhir tahun. Penurunan kinerja pada September sampai Oktober menjadi indikator penting untuk memperkuat aktivitas pemasaran di Q3. Konsistensi antara transaksi dan revenue menunjukkan strategi harga yang sehat. Secara keseluruhan, peluang pertumbuhan 2025 dapat dioptimalkan melalui seasonal campaign yang lebih terstruktur, penguatan promosi di Q3, ekspansi layanan delivery, dan peningkatan program loyalitas berbasis data.

## **Area 2 - Product**

_Jenis pizza apa saja yang paling banyak dipesan berdasarkan data transaksi DQPizza? Tampilkan top 10 pizza_

In [ ]:

%%bigquery total_order_per_pizzaname --project {project_id} --verbose

SELECT
  -- 2. Menggabungkan kategori dan nama pizza dengan pemisah '|'
  CONCAT(pizza_category, ' | ', pizza_name) AS pizza_name,

  -- 3. Menghitung jumlah TRANSAKSI / ORDER UNIK
  COUNT(DISTINCT order_id) AS total_order
FROM
  -- 1. Menggunakan data transaksi penjualan pizza
  dq_pizza.tbl_all_transaction
GROUP BY
  -- Mengelompokkan berdasarkan nama dan kategori
  pizza_category,
  pizza_name
ORDER BY
  -- 4. Mengurutkan hasil dari yang paling banyak dipesan
  total_order DESC
LIMIT 10; -- 5. Menampilkan hanya 10 jenis pizza teratas

In [ ]:
# Tampilkan hasilnya
display(total_order_per_pizzaname)

In [ ]:
#@title ***Total Order per Pizza Name*** {display-mode: 'form'}

import plotly.express as px

total_order_per_pizzaname.sort_values(
    by = 'total_order',
    ascending = True,
    ignore_index = True,
    inplace = True
)

total_order_per_pizzaname['pizza_name'] = total_order_per_pizzaname['pizza_name'].str.title()
total_order_per_pizzaname['pizza_name'] = '' + total_order_per_pizzaname['pizza_name'].str.replace(' |', ' |')
total_order_per_pizzaname['Color'] = total_order_per_pizzaname['total_order'].apply(
    lambda x: 'Top' if x == total_order_per_pizzaname['total_order'].max() else 'Other'
)

fig = px.bar(
    total_order_per_pizzaname,
    x = 'total_order',
    y = 'pizza_name',
    orientation = 'h',
    color = 'Color',
    color_discrete_map = {
        'Other': '#85ff7a',
        'Top': '#2db83d'
    },
    text_auto = True
)

fig.update_layout(
    width = 950,
    height = 450,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = 'Best Seller Pizza Berdasarkan NamaPeriode 2024',
        font = dict(color = '#D19C4B')
    )
)

fig.update_xaxes(showticklabels = False)

fig.update_traces(
    textposition = 'inside',
    hovertemplate = 'Pizza %{label}Total Order = %{value}'
)

fig.show()

**Kesimpulan :**
Menu yang paling best seller adalah Classic Deluxe Pizza, kemudian untuk kategori Classic dan Chicken adalah kategori favorit konsumen. selain itu, terdapat 3 menu yang konsisten rendah penjualannya dan mungkin butuh strategi perbaikan. secara keseluruhan, distribusi penjualan cukup merata dengan sedikit outlier di bagian bawah.

## **Area 3 - Service**

_Pada jam berapa pembuatan pizza di DQPizza paling banyak dilakukan di setiap hari dalam seminggu berdasarkan data?_

In [ ]:
%%bigquery num_qty_per_hour --project {project_id} --verbose

WITH
DailyHourlySum AS (
  SELECT
    order_date,
    EXTRACT(HOUR FROM order_time) AS jam,
    SUM(quantity) AS total_pizza_harian_per_jam
  FROM
    `dq_pizza.tbl_all_transaction`
  GROUP BY
    1, 2
),
AverageHourlyStats AS (
  SELECT
    FORMAT_DATE('%A', order_date) AS nama_hari,
    jam,
    AVG(total_pizza_harian_per_jam) AS rata_rata_pizza_per_jam_slot
  FROM
    DailyHourlySum
  GROUP BY
    1, 2
)
SELECT
  nama_hari AS day_trx,
  jam AS hour_trx,
  ROUND(rata_rata_pizza_per_jam_slot, 2) AS avg_quantity
FROM
  AverageHourlyStats
ORDER BY
  avg_quantity DESC;


In [ ]:
display(num_qty_per_hour)

In [ ]:
#@title ***Rata - Rata Pizza Dibuat Tiap Jam Setiap Harinya*** {display-mode: 'form'}

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

avg_qty_per_hour = num_qty_per_hour.groupby(['hour_trx'], as_index = False).agg(avg_qty_per_hour = ('avg_quantity', 'mean'))

day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
num_qty_per_hour['day_trx'] = pd.Categorical(num_qty_per_hour['day_trx'], categories=day_order, ordered=True)

num_qty_per_hour_pivot = num_qty_per_hour.pivot(
    index = 'day_trx',
    columns = 'hour_trx',
    values = 'avg_quantity'
).sort_values(by = 'day_trx', ascending = False, ignore_index = False)

fig = make_subplots(
    rows = 2,
    cols = 1,
    shared_xaxes = True,
    row_heights = [0.25, 0.75],
    vertical_spacing = 0.01
)

fig.add_trace(
    px.bar(
        avg_qty_per_hour,
        x = 'hour_trx',
        y = 'avg_qty_per_hour',
        labels = dict(
            x = 'month_trx',
            y = 'total_revenue'
        ),
        color = 'avg_qty_per_hour',
    ).data[0],
    row = 1,
    col = 1
)

fig.update_layout(
    yaxis = dict(
        showline = False,
        showgrid = False,
        showticklabels = False,
    )
)

fig.add_trace(
    px.imshow(
        num_qty_per_hour_pivot
    ).data[0],
    row = 2,
    col = 1
)

fig.update(
    layout_coloraxis_showscale = False
)

fig.update_xaxes(
    automargin = True
)

fig.update_layout(
    height = 600,
    width = 700,
    bargap = 0.05,
    coloraxis_colorscale = 'oranges',
    xaxis_tickangle = 0,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b>Rata - Rata Pizza Dibuat Tiap Jam Setiap Harinya</b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(
            color = '#B54B1F'
        )
    )
)

fig.show()

**Kesimpulan :**

berdasarkan visualisasi data yang ditampilkan,didapatkan bahwa dua periode puncak aktivitas dapur terjadi pada 11.00 sampai 13.00 dan 17.00 sampai 19.00 memiliki intensitas tertinggi pada Selasa, Rabu, Sabtu, dan Minggu. Di luar jam tersebut—terutama pagi hari dan setelah 20.00 aktivitas menurun sehingga waktu tersebut ideal untuk persiapan.

Setiap hari memiliki ritme berbeda: Selasa dan Rabu konsisten ramai hingga malam, sementara Sabtu dan Minggu padat di tengah hari. Monday dan Thursday relatif landai. Pola ini menjadi dasar penyesuaian jumlah staf dan stok, di mana hari-hari sibuk memerlukan tambahan tenaga dan peningkatan bahan baku hingga 10% hingga 25%.

Rekomendasi penjadwalan mengikuti kebutuhan operasional: penguatan tim pada shift siang dan sore, serta alokasi minimal pada malam hari. Untuk menjaga kelancaran produksi, disarankan melakukan pre-baking dough sebelum puncak siang, batch preparation topping sebelum puncak sore, serta memastikan fleksibilitas tim melalui cross-training. Dengan pendekatan ini, dapur dapat beroperasi lebih efisien dan responsif terhadap permintaan harian.

_Bagaimana perbandingan jumlah pesanan yang disertai keluhan (complain) dan yang tidak disertai keluhan berdasarkan data?_

In [ ]:
%%bigquery proportion_complain --project {project_id} --verbose

SELECT
  CASE
    WHEN is_complain = 1 THEN 'Complain'
    ELSE 'No Complain'
  END AS is_complain_label,
  COUNT(*) AS total_complain
FROM
  dq_pizza.tbl_all_transaction
GROUP BY
  is_complain_label;

In [ ]:
display(proportion_complain)

In [ ]:
#@title ***Proporsi Kejadian Komplain di DQPizza*** {display-mode: 'form'}

# Import library untuk visualisasi
import plotly.express as px

# Hitung total data
total_data = proportion_complain['total_complain'].sum()

# Warna
hijau_pucat = '#E0ECE4'
merah = '#FF4B5C'

# Buat pie chart
fig = px.pie(
    values = proportion_complain['total_complain'],
    names = proportion_complain['is_complain_label'],
    color_discrete_sequence = [hijau_pucat, merah],
    hole = 0.65
)

# Atur posisi label
fig.update_traces(
    textposition = 'outside',
    textinfo = 'percent+label',
    hovertemplate='%{label}%{value} Customers'
)

# Atur luas grafik, hapus legend dan beri judul
fig.update_layout(
    width = 800,
    height = 600,
    showlegend = False,
    margin = dict(l=160, r=200, t=100, b=30),
    title = dict(
        text = f"Proporsi Kejadian Komplain di DQPizza",
        font = dict(
            size = 25,
            color = '#757882'
        ),
        y = 0.92,
        x = 0.46
    )
)

# Berikan informasi total pelanggan di tengah donut chart
fig.add_annotation(
    text = f'Total Transaksi{total_data}',
    x = 0.5,
    y = 0.5,
    showarrow = False,
    font = dict(size = 30)
)

# Tampilkan grafik
fig.show()

**Kesimpulan :**

Dari total 48.619 transaksi, sekitar 10% tercatat sebagai komplain. Meskipun persentasenya relatif kecil, jumlah ini tetap perlu dianalisis lebih lanjut untuk mengetahui sumber utama keluhan. Langkah pertama adalah mengidentifikasi kategori komplain yang paling sering muncul—seperti keterlambatan pengiriman, kualitas pizza yang tidak konsisten, pesanan yang salah, atau masalah pengemasan. Selanjutnya, penting untuk melihat pola waktu dan hari terjadinya komplain, apakah lebih tinggi pada jam-jam puncak atau hari dengan volume pesanan yang tinggi, karena hal ini dapat mengindikasikan adanya tekanan operasional yang memengaruhi kualitas layanan. Evaluasi terhadap proses dapur dan pengiriman juga perlu dilakukan untuk menilai ketepatan waktu, akurasi pesanan, serta konsistensi kualitas produk. Selain itu, analisis umpan balik pelanggan dapat membantu mengidentifikasi masalah spesifik pada menu atau alur pelayanan. Melalui temuan ini, perbaikan dapat diarahkan pada peningkatan SOP produksi, penambahan staf di waktu kritis, optimalisasi alur kerja dapur, dan penguatan kualitas layanan secara keseluruhan.

_Jenis keluhan apa saja yang paling sering muncul pada pesanan pelanggan berdasarkan data transaksi?_

In [ ]:
%%bigquery total_detail_complain --project {project_id} --verbose

SELECT
  complain_detail,
  COUNT(*) AS jumlah_complain
FROM
  dq_pizza.tbl_all_transaction
WHERE
  is_complain = 1 AND complain_detail IS NOT NULL
GROUP BY
  complain_detail
ORDER BY
  jumlah_complain ASC;

In [ ]:
display(total_detail_complain)

In [ ]:
#@title ***Detail Komplain yang Terjadi di DQPizza*** {display-mode: 'form'}

import plotly.express as px

total_detail_complain['Color'] = total_detail_complain['jumlah_complain'].apply(
    lambda x: 'Top' if x == total_detail_complain['jumlah_complain'].max() else 'Other'
)

fig = px.bar(
    total_detail_complain,
    x = 'jumlah_complain',
    y = 'complain_detail',
    orientation = 'h',
    color = 'Color',
    color_discrete_map = {
        'Other': '#ffb3b3',
        'Top': '#ff0000'
    },
    text_auto = True
)

fig.update_layout(
    width = 950,
    height = 450,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b>Detail Komplain yang Terjadi di DQPizza</b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(color = '#ff0000')
    )
)

fig.update_xaxes(showticklabels = False)

fig.update_traces(
    textposition = 'inside',
    hovertemplate = '<b>Pizza %{label}</b><br>Total Order = %{value}'
)

fig.show()

**Kesimpulan :**

Hasil analisis komplain menunjukkan bahwa sebagian besar keluhan pelanggan berasal dari masalah operasional yang dapat dikendalikan oleh dapur maupun frontliner. Keluhan paling tinggi adalah topping yang tidak lengkap, disusul oleh pizza gosong, yang keduanya menunjukkan perlunya peningkatan kontrol kualitas pada tahap produksi. Selain itu, jumlah komplain terkait pesanan terlambat, pizza salah, dan ukuran yang tidak sesuai menandakan perlunya perbaikan pada koordinasi antara dapur dan bagian pengantaran, terutama pada jam-jam sibuk. Keluhan lain seperti minuman tidak tersertakan, dough kurang matang, serta insiden terkait pelayanan staf juga menyoroti pentingnya standardisasi SOP dan peningkatan konsistensi kerja tim. Secara keseluruhan, temuan ini memberikan gambaran jelas mengenai area prioritas yang perlu ditingkatkan, baik dari sisi kualitas produk, ketepatan operasional, maupun pengalaman pelanggan.

_Bagaimana distribusi dan jenis keluhan pelanggan yang diterima oleh masing-masing pembuat pizza (order maker) berdasarkan data transaksi?_

In [ ]:
%%bigquery total_complain_per_ordermaker --project {project_id} --verbose

SELECT
  order_maker_id,
  SUM(CASE WHEN complain_detail = 'Dough undercooked' THEN 1 ELSE 0 END) AS Dough_undercooked,
  SUM(CASE WHEN complain_detail = 'Drink was missing' THEN 1 ELSE 0 END) AS Drink_was_missing,
  SUM(CASE WHEN complain_detail = 'Missing toppings' THEN 1 ELSE 0 END) AS Missing_toppings,
  SUM(CASE WHEN complain_detail = 'Order took too long' THEN 1 ELSE 0 END) AS Order_took_too_long,
  SUM(CASE WHEN complain_detail = 'Pizza was burnt' THEN 1 ELSE 0 END) AS Pizza_was_burnt,
  SUM(CASE WHEN complain_detail = 'Received wrong pizza' THEN 1 ELSE 0 END) AS Received_wrong_pizza,
  SUM(CASE WHEN complain_detail = 'Rude staff at counter' THEN 1 ELSE 0 END) AS Rude_staff_at_counter,
  SUM(CASE WHEN complain_detail = 'Wrong pizza size' THEN 1 ELSE 0 END) AS Wrong_pizza_size
FROM
  dq_pizza.tbl_all_transaction
WHERE
  is_complain = 1
GROUP BY
  order_maker_id
ORDER BY
  order_maker_id;

In [ ]:
display(total_complain_per_ordermaker)

In [ ]:
#@title ***Jumlah Detail Komplain per Pizza Order Maker*** {display-mode: 'form'}

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = px.imshow(
    total_complain_per_ordermaker.set_index('order_maker_id'),
    text_auto = True,
)

fig.update(
    layout_coloraxis_showscale = False
)

fig.update_layout(
    height = 1000,
    width = 800,
    coloraxis_colorscale = 'reds',
    xaxis_tickangle = 0,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b>Jumlah Detail Komplain per <i>Pizza Order Maker</i></b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(
            color = '#B54B1F',
            size = 20
        )
    ),
    margin=dict(t=150)
)

fig.update_xaxes(
    tickangle = -15,
    side = 'top',
    automargin = True,
    title = None,
    tickfont = dict(size=11)
)

fig.update_yaxes(
    title = None,
    tickfont = dict(size=11)
)

fig.show()

**Kesimpulan :**

Analisis komplain per pizza order maker menunjukkan adanya ketidakkonsistenan kinerja yang berdampak langsung pada kualitas produk. Beberapa karyawan tercatat menerima komplain berulang pada kategori yang sama seperti kesalahan topping, tingkat kematangan yang tidak sesuai, dan keterlambatan produksi yang mengindikasikan pelanggaran prosedur operasional standar. Temuan ini menegaskan perlunya tindakan korektif yang terarah, mulai dari coaching individu, penegakan disiplin proses, hingga pelatihan ulang pada area kritis. Dengan intervensi yang tepat, perusahaan dapat meningkatkan akurasi produksi, menurunkan tingkat komplain, dan memastikan standar kualitas tetap konsisten di seluruh lini operasional.

## **Area 4 - Customer**

_Bagaimana distribusi usia pelanggan berdasarkan kategori gender?_

In [ ]:
%%bigquery age_and_gender --project {project_id} --verbose

SELECT
  customer_gender,
  customer_age
FROM
  `dq_pizza.tbl_all_transaction`
WHERE customer_gender IS NOT NULL AND customer_age IS NOT NULL


In [ ]:
display(age_and_gender)

In [ ]:
# @title ***Distribusi Usia Pelanggan*** {display-mode: 'form'}

# Import library yang dibutuhkan
import plotly.figure_factory as ff

def distribution_plot(data_cat_1, data_cat_2, label, column_name):
    # Group data together
    data_cat_1 = data_cat_1[column_name]
    data_cat_2 = data_cat_2[column_name]
    hist_data = [data_cat_1, data_cat_2]
    group_labels = label

    # Create distplot with custom bin_size
    fig = ff.create_distplot(
        hist_data,
        group_labels,
        show_hist = False,
        show_rug = False
    )

    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        title = dict(
            text = f"<b>Distribusi Usia Pelanggan</b>",
            font = dict(
                size = 28,
                color = 'black'
            ),
            y = 0.92,
            x = 0.5
        )
    )

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')

    # Tampilkan visualisasi
    fig.show()

male_cust = age_and_gender[age_and_gender['customer_gender'] == 'M']
female_cust = age_and_gender[age_and_gender['customer_gender'] == 'F']
label_grup =  ['Male', 'Female']

distribution_plot(male_cust, female_cust, label_grup, 'customer_age')

**Kesimpulan :**

Distribusi usia menunjukkan perbedaan preferensi dan perilaku antara pelanggan laki-laki dan perempuan. Dengan memanfaatkan pola ini, bisnis dapat mengoptimalkan promosi, menciptakan menu khusus, dan meningkatkan efektivitas pemasaran secara signifikan.

_Bagaimana preferensi kategori pizza berdasarkan gender pelanggan pada data transaksi?_

In [ ]:
%%bigquery pizza_category_preferences --project {project_id} --verbose

SELECT
  customer_gender,
  pizza_category,
  COUNT(*) AS total_pizza_category
FROM
  dq_pizza.tbl_all_transaction
WHERE customer_gender IS NOT NULL AND pizza_category IS NOT NULL
GROUP BY
  customer_gender,
  pizza_category
ORDER BY
  customer_gender,
  total_pizza_category DESC;

In [ ]:
display(pizza_category_preferences)

In [ ]:
#@title ***Preferensi Kategori Pizza per Order*** {display-mode: 'form'}

import pandas as pd
import numpy as np
import plotly.express as px

pivot = pizza_category_preferences.pivot_table(
    index='pizza_category',
    columns='customer_gender',
    values='total_pizza_category',
    aggfunc='sum',
    fill_value=0
).reset_index()

gender_cols = [c for c in pivot.columns if c != 'pizza_category']
if len(gender_cols) < 2:
    raise ValueError("Butuh minimal dua kategori gender untuk butterfly chart.")
g1, g2 = gender_cols[0], gender_cols[1]

pivot[f'{g1}_neg'] = -pivot[g1]

fig = px.bar(
    pivot,
    y='pizza_category',
    x=[f'{g1}_neg', g2],
    orientation='h',
    text_auto=True,
    title=f'<b>Preferensi Kategori Pizza per Gender: {g1} vs {g2}</b>',
    labels={'pizza_category': 'Pizza Category'}
)

max_val = int(max(pivot[g1].max(), pivot[g2].max()))
xticks = np.linspace(-max_val, max_val, 9)
xticktext = [str(abs(int(x))) for x in xticks]

fig.update_layout(
    yaxis_title=None,
    barmode='relative',
    bargap=0.2,
    height=700,
    width=1000,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(
        tickmode='array',
        tickvals=xticks,
        ticktext=xticktext,
        title='Jumlah Pesanan'
    ),
    legend_title_text='Gender'
)

fig.update_xaxes(showticklabels=False)
fig.update_traces(texttemplate='%{text}', textposition='inside')

# --- Ubah nama trace agar legend tampil rapi (tanpa _neg) ---
fig.data[0].name = g1
fig.data[1].name = g2

# --- Pastikan nilai teks selalu positif ---
for trace in fig.data:
    vals = np.abs(trace.x)
    trace.text = vals
    trace.textfont = dict(color='white', size=11)

fig.add_vline(x=0, line_width=1, line_color='black')

fig.show()

**Kesimpulan :**
Dari hasil segmentasi kampanye, pelanggan laki-laki menjadi target yang sangat potensial untuk empat kategori sekaligus yaitu Classic, Supreme, Chicken, dan Veggie. Kampanye yang efektif untuk segmen ini mencakup visual bold, sporty, dan nuansa gaming, dipadukan dengan promosi seperti Buy 1 Get 1 for Men’s Night serta paket sharing untuk momen nonton bola. Platform seperti YouTube, TikTok (segmen gaming dan olahraga), serta Instagram Reels menjadi kanal yang tepat. Sementara itu, untuk pelanggan perempuan, kategori yang paling sesuai adalah Classic, Veggie, dan Chicken karena segmentasi rasanya lebih aman dan ringan. Kampanye dapat menggunakan visual clean dengan warna cerah atau pastel, serta menawarkan promosi seperti Ladies Special Discount, Healthy Choice Package (Veggie + minuman rendah gula), dan Classic for Two untuk target pasangan atau sahabat. Platform utama untuk menjangkau segmen perempuan antara lain Instagram Feed & Stories, TikTok dengan tema lifestyle atau food review, serta marketplace ads seperti GoFood dan GrabFood.

Data Source : <i>https://mavenanalytics.io/challenges/maven-pizza-challenge</i> (dengan modifikasi)


---

<br>
<a href="https://www.linkedin.com/in/muhamad-rasyid-aditya-17637a225/"><img src="https://img.shields.io/badge/-© 2025 Muhamad Rasyid Aditya-417DAC?style=for-the-badge&logoColor=white"/></a>

<a href="https://dqlab.id/"><img src="https://dqlab.id/files/dqlab/cache/87e30118ebba5ec7d96f6ea8c9dcc10b_x_118_X_55.png" align="left" /></a>
